In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("sweatSmile/marx-opt350m-finetuned-v1")
tokenizer = AutoTokenizer.from_pretrained("sweatSmile/marx-opt350m-finetuned-v1")

prompt = "### Human: Who were commissioned to prepare the party programme for the Communist League?\n### Assistant:"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# Define a list of questions
questions = [
    "Who were commissioned to prepare the party programme for the Communist League?",
    "What is surplus value according to Karl Marx?",
    "Explain the concept of class struggle in Marxist economics.",
    "How does Marx define capitalism?",
    "What is the difference between use value and exchange value?",
    "Can you explain Karl Marx's theory of historical materialism?",
    "What role does labor play in value creation, according to Marx?",
    "How does Marxist theory explain economic crises?",
    "What is the Marxist view of alienation?",
    "How would Marx critique modern-day corporate capitalism?"
]

# Iterate through questions and get responses
for question in questions:
    prompt = f"### Human: {question}\n### Assistant:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=150, do_sample=True, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"🧠 Q: {question}\n💬 A: {response.split('### Assistant:')[-1].strip()}\n{'-'*80}")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load original OPT-350M model and tokenizer
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

# Generate and print answers
for question in questions:
    prompt = f"### Human: {question}\n### Assistant:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=150, do_sample=True, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"🧠 Q: {question}\n💬 A: {response.split('### Assistant:')[-1].strip()}\n{'-'*80}")


In [ ]:
pip install tabulate


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tabulate import tabulate
import textwrap

# Define your questions
questions = [
    "Who were commissioned to prepare the party programme for the Communist League?",
    "What is surplus value according to Karl Marx?",
    "Explain the concept of class struggle in Marxist economics.",
    "How does Marx define capitalism?",
    "What is the difference between use value and exchange value?",
]

# Load models
models = {
    "SweatSmile Fine-Tuned": {
        "model": AutoModelForCausalLM.from_pretrained("sweatSmile/marx-opt350m-finetuned-v1"),
        "tokenizer": AutoTokenizer.from_pretrained("sweatSmile/marx-opt350m-finetuned-v1")
    },
    "Facebook OPT-350M Base": {
        "model": AutoModelForCausalLM.from_pretrained("facebook/opt-350m"),
        "tokenizer": AutoTokenizer.from_pretrained("facebook/opt-350m")
    }
}

# Wrap text to fit better in the table
def wrap(text, width=60):
    return "\n".join(textwrap.wrap(text, width=width))

# Collect responses
results = []
for q in questions:
    row = [wrap(q)]
    for name, parts in models.items():
        inputs = parts["tokenizer"](f"### Human: {q}\n### Assistant:", return_tensors="pt")
        outputs = parts["model"].generate(**inputs, max_length=150, do_sample=True, temperature=0.7)
        response = parts["tokenizer"].decode(outputs[0], skip_special_tokens=True)
        answer = response.split("### Assistant:")[-1].strip()
        row.append(wrap(answer))
    results.append(row)

# Display clean table
headers = ["Question", "SweatSmile Fine-Tuned", "Facebook OPT-350M Base"]
print(tabulate(results, headers=headers, tablefmt="fancy_grid"))


In [ ]:
# To save as markdown
with open("model_comparison.md", "w") as f:
    f.write(tabulate(results, headers=headers, tablefmt="github"))

# To save as CSV
import pandas as pd
df = pd.DataFrame(results, columns=headers)
df.to_csv("model_comparison.csv", index=False)